In [ ]:
import numpy as np
import torch
import torchio.transforms
from tqdm import tqdm
import os
import glob
import SimpleITK as sitk
from scipy import ndimage
import copy
import random
import time
import numpy as np
import torch
from PIL import Image
from scipy.special import comb
from torch.utils.data import Dataset


## 其他

测试CUDA

In [ ]:
#测试cuda
import torch
print(torch.__version__)
print(torch.cuda.is_available())

查看permutations文件

In [ ]:
k_permutations_path = r'datasets_3D\Rubik_cube\permutaions\permutations_hamming_max_100.npy'
K_permutations = np.load(k_permutations_path)
#可以用vscode自带的数据查看器查看

测试np.flip产生的错误step must be greater than zero

In [ ]:
x = torch.randn(10, 10)
# x[:, ::-1]
# > ValueError: step must be greater than zero
x.numpy()[:, ::-1] # works


In [ ]:

a = np.random.randn(10, 10)
a.strides
# > (80, 8)


In [ ]:
a = np.flip(a)
a.strides
# > (-80, -8) strides的涵义我收藏了一篇文章可以翻阅

x = torch.from_numpy(a)
# > ValueError: At least one stride in the given numpy array is negative, and tensors with negative strides are not currently supported. (You can probably work around this by making a copy of your array  with array.copy().) 

x = torch.from_numpy(a.copy()) # works
x.stride()
# > (10, 1)

## 测试MRI_rkb_plus_pretask中的get_item

### 测试全代码

设置假设input

In [ ]:
t1 = torch.rand(240,240,155)
print(t1.shape)

设置假设config参数值

In [ ]:
crop_size = [120, 120, 77]
flag = 'train'
num_grids_per_axis = 2
num_cubes = num_grids_per_axis ** 3
print(num_cubes)
k_permutations_path = r'datasets_3D\Rubik_cube\permutaions\permutations_hamming_max_100.npy'
K_permutations = np.load(k_permutations_path)


def会用到的一些函数（from base_rkb_pretask）

In [ ]:
def crop_3d(image, flag, crop_size): # ...（对图像执行3D裁剪的方法）
    h, w, d = crop_size[0], crop_size[1], crop_size[2]
    h_old, w_old, d_old = image.shape[0], image.shape[1], image.shape[2]

    if flag == 'train': #如果 flag 是 'train'，则进行随机裁剪。在这种情况下，生成随机的裁剪起始点 (x, y, z)，h_old-h是为了确保裁剪框不超出图像边界。
        # crop random
        x = np.random.randint(0, 1 + h_old - h) #返回一个随机数或随机数数组(指定size时)
        y = np.random.randint(0, 1 + w_old - w)
        z = np.random.randint(0, 1 + d_old - d)
    else: #如果 flag 不是 'train'，则进行中心裁剪。计算裁剪起始点 (x, y, z) 使得裁剪的区域在图像中居中。
        # crop center
        x = int((h_old - h) / 2)
        y = int((w_old - w) / 2)
        z = int((d_old - d) / 2)

    return do_crop_3d(image, x, y, z, h, w, d)

def do_crop_3d(image, x, y, z, h, w, d): #3D裁剪的辅助函数，确保整数和返回对应位置的3D块
    assert type(x) == int, x
    assert type(y) == int, y
    assert type(z) == int, z
    assert type(h) == int, h
    assert type(w) == int, w
    assert type(d) == int, d

    return image[x:x + h, y:y + w, z:z + d] #这边取的时最边界点加上crop_size的大小，解释了为何上述代码的定中心方式如此

def crop_cubes_3d(image, flag, cubes_per_side, cube_jitter_xy=3, cube_jitter_z=3): #在3D图像中裁剪多个3D立方体的方法，jitter貌似是为了防止数据连续变化留出的间隔大小，从代码里可以看出如果有jitter则再次进行3D裁切，cubes_per_side应该是每个方向有多少个cube
    h, w, d = image.shape

    patch_overlap = -cube_jitter_xy if cube_jitter_xy < 0 else 0 #patch_overlap 计算了在裁剪时可能存在的重叠区域。如果 cube_jitter_xy 是负数，则 patch_overlap 采用该值；否则，为零。

    #h_grid、w_grid 和 d_grid 计算了每个立方体的网格大小，以确保裁剪时没有重叠区域。
    # 这里主要思想应该是把切出来的3D大块划分为各个小块,这里拿笔记做做数学题吧，好久没写了。
    h_grid = (h - patch_overlap) // cubes_per_side
    w_grid = (w - patch_overlap) // cubes_per_side
    d_grid = (d - patch_overlap) // cubes_per_side
    h_patch = h_grid - cube_jitter_xy
    w_patch = w_grid - cube_jitter_xy
    d_patch = d_grid - cube_jitter_z

    cubes = []
    for i in range(cubes_per_side):
        for j in range(cubes_per_side):
            for k in range(cubes_per_side):

                p = do_crop_3d(image, #当i=0时，从0截取到第一个grid
                                i * h_grid,
                                j * w_grid,
                                k * d_grid,
                                h_grid + patch_overlap,
                                w_grid + patch_overlap,
                                d_grid + patch_overlap)

                if h_patch < h_grid or w_patch < w_grid or d_patch < d_grid: #如果有jitter，则对之前裁切好的3D块再次裁切以保留jitter大小的间隔
                    p = crop_3d(p, flag, [h_patch, w_patch, d_patch])

                cubes.append(p)

    return cubes

def rearrange(cubes, K_permutations):  # ...根据排列重新排列立方体的方法
    label = random.randint(0, len(K_permutations) - 1)
    print('label', np.array(K_permutations[label]), label)
    return np.array(cubes)[np.array(K_permutations[label])], label

def center_crop_xy(image, size): # 在image中间截一个size大小的缺口，画图很容易理解
    """CenterCrop a sample.
        Args:
            image: [D, H, W]
            label:[D, H, W]
            crop_size: the desired output size in the x-y plane
        Returns:
            out_image:[D, h, w]
            out_label:[D, h, w]
    """
    h, w, d = image.shape

    h1 = int(round((h - size[0]) / 2.)) #round() 函数将结果四舍五入为最接近的整数，确保裁剪区域的起始点是整数。
    w1 = int(round((w - size[1]) / 2.))

    image = image[h1:h1 + size[0], w1:w1 + size[1], :]
    return image

def rotate(cubes): # ...（旋转3D立方体的方法）

    # multi-hot labels
    # [8, H, W, D]
    rot_cubes = copy.deepcopy(cubes) # 创建输入立方体的深层副本，以免修改原始数据
    hor_vector = [] # 记录水平旋转的向量
    ver_vector = [] # 记录垂直旋转的向量

    for i in range(num_cubes):
        p = random.random()  # 生成一个0到1之间的随机数
        cube = rot_cubes[i] # 获取当前处理的立方体,这里debug看cube是个tensor，后续会出bug
        cube_ = cube.numpy() #设置一个numpy_arrary防止np.flip bug
        # p = 0.1 # for test
        # [H, W, D]
        if p < 1/3: # 如果 p 小于 1/3，表示进行水平旋转。将水平旋转的标志添加到 hor_vector，并沿x轴翻转180度。
            hor_vector.append(1)
            ver_vector.append(0)
            # rotate 180 along x axis
            # rot_cubes[i] = np.flip(cube, (1, 2)) # 沿x轴翻转180度,BUG ValueError: step must be greater than zero,最后输出的应该是numpy形式
            cube = np.flip(cube_, (1, 2))
            rot_cubes[i] = cube
        elif p < 2/3: #如果 p 大于等于 1/3 且小于 2/3，表示进行垂直旋转。将垂直旋转的标志添加到 ver_vector，并沿z轴翻转180度。
            hor_vector.append(0)
            ver_vector.append(1)
            # rotate 180 along z axis
            # rot_cubes[i] = np.flip(cube, (0, 1)) # 沿z轴翻转180度
            cube = np.flip(cube_, (0, 1))
            rot_cubes[i] = cube

        else: #如果 p 大于等于 2/3，不进行旋转，标志都设为0。
            hor_vector.append(0)
            ver_vector.append(0)

    return rot_cubes, hor_vector, ver_vector

def mask(cubes): # ...（对3D立方体应用掩码的方法）
    mask_vector = []
    masked_cubes = copy.deepcopy(cubes) # 创建一个输入立方体的深层副本，以免修改原始数据
    for i in range(num_cubes):
        cube = masked_cubes[i] # 获取当前处理的立方体
        if random.random() < 0.5: # 如果随机数小于0.5，应用掩码
            # mask
            mask_vector.append(1) # 记录掩码的标志为1
            R = np.random.uniform(0, 1, cube.shape) # 生成与立方体相同形状的随机数矩阵
            R = (R > 0.5).astype(np.int32) # 将大于0.5的值设置为1，否则为0，形成二值掩码
            masked_cubes[i] = cube * R # 将立方体与二值掩码相乘，进行掩码操作
        else:
            mask_vector.append(0) # 如果随机数大于等于0.5，不应用掩码，标志为0

    return masked_cubes, mask_vector

测试代码主体

In [ ]:
input_tensor = torch.Tensor(t1)
# input: [240, 240, 155]
        
input = input_tensor[:,:,0:154]
# 截去最后一层方便分块整除

if crop_size == [128, 128, 32]:
    # input: [276, 276, 74]
    input = center_crop_xy(input, [276, 276])

    # get all the num_grids **3 cubes
    all_cubes = crop_cubes_3d(
        input,
        flag=flag,
        cubes_per_side=num_grids_per_axis,
        cube_jitter_xy=10,
        cube_jitter_z=5,
    )
    print(len(all_cubes), all_cubes[0].shape)

# 这部分添加适合MRI的块大小
# MRI数据集大小[240,240,155],为了能够整除，应该截成154的depth，设crop_size=[120，120，77],之所以做的是整除原因在于抄的上面的代码也是整除
elif crop_size == [120, 120, 77]:
    # input: [276, 276, 74]
    # input = center_crop_xy(input, [276, 276])

    # get all the num_grids **3 cubes
    all_cubes = crop_cubes_3d(
        input,
        flag=flag,
        cubes_per_side=num_grids_per_axis,
        cube_jitter_xy=10,
        cube_jitter_z=5,
    )
    print(len(all_cubes), all_cubes[0].shape)

elif crop_size == [64, 64, 16]:
    # input: [140, 140, 40]
    input = ndimage.zoom(input, [140 / 320, 140 / 320, 40 / 74], order=3)

    # get all the num_grids **3 cubes
    all_cubes = crop_cubes_3d(
        input,
        flag=flag,
        cubes_per_side=num_grids_per_axis,
        cube_jitter_xy=6,
        cube_jitter_z=4,
    )

else:
    NotImplementedError("This crop size has not been configured yet")
    all_cubes = None

# 这里all_cubes还是list形式

In [ ]:

# Task1: Permutate the order of cubes
rearranged_cubes, order_label = rearrange(all_cubes, K_permutations)

print(rearranged_cubes.dtype)

In [ ]:
# Task2: Rotate each cube randomly.
rotated_cubes, hor_label, ver_label = rotate(rearranged_cubes)

# Task2: Mask each cube randomly.
masked_cubes, mask_label = mask(rotated_cubes)

print(rotated_cubes.shape) # (8,)
print(rotated_cubes[0].shape) # torch.Size([110, 110, 72])



In [ ]:
# final_cubes = np.expand_dims(np.array(masked_cubes), axis=1) #在a xis=1的位置增加一个维度，可参文章
final_cubes = masked_cubes.tolist() # 转为list方便转换为tensor
final_cubes_tensors = torch.stack(torch.from_numpy(final_cubes))


In [ ]:

print(final_cubes.shape) # (8, 1)
print(final_cubes[0].shape) # (1,)
print(final_cubes[0][0].shape) # torch.Size([110, 110, 72])
print(final_cubes_tensors.shape) # (8, 1)
print(final_cubes_tensors[0].shape) # (1,)
print(final_cubes_tensors[0][0].shape) # torch.Size([110, 110, 72])


In [ ]:
final_cubes_tensors = torch.stack(cubes)
print(final_cubes_tensors.dtype)

In [ ]:
final_cubes_list = final_cubes.tolist()
final_cubes_tensors = torch.stack(final_cubes_list)
print(final_cubes_tensors.dtype)

return的结果

In [ ]:
# 以下是4个要输出的值
print(torch.from_numpy(final_cubes.astype(np.float32)))
print(torch.from_numpy(np.array(order_label)))
print(torch.from_numpy(np.array(hor_label)).float())
print(torch.from_numpy(np.array(ver_label)).float())


### 测试rearrange函数

问题在rearrange函数，进行调试

In [ ]:
# 创建包含8个随机张量的张量
num_tensors = 8
tensor_size = [110, 110, 72]

random_tensors = [torch.rand(tensor_size) for _ in range(num_tensors)]
# tensor_arry = np.array(random_tensors) # 源代码中输入的就是list，不需要转为nparrary


# 打印结果
print(len(random_tensors))
print(random_tensors[0].shape) # (1,)


In [ ]:
cubes = random_tensors
cubes_tensor = torch.stack(cubes) # 将嵌套list转为tensor增加测试
k_permutations_path = r'datasets_3D\Rubik_cube\permutaions\permutations_hamming_max_100.npy'
K_permutations = np.load(k_permutations_path)

In [ ]:
label = random.randint(0, len(K_permutations) - 1)
label = 50 # 设置个label方便debug

In [ ]:
choosed_permutation = K_permutations[label] # 这里挑出来是nparray类型
print(choosed_permutation)
print(len(choosed_permutation))

测试tensor排序

In [ ]:
rearrange_cubes_tensor = cubes_tensor[choosed_permutation]
# 检查是否相等
are_close = torch.allclose(rearrange_cubes_tensor[2], cubes_tensor[0])
print(are_close)
are_close = torch.allclose(rearrange_cubes_tensor[1], cubes_tensor[0])
print(are_close)

In [ ]:
cubes_array = np.array(cubes) # 这句报警告

In [ ]:
cubes_array

In [ ]:
cubes_rearrange =cubes_array[np.array(K_permutations[label])] # 貌似用nparray操作最合适不会报错

In [ ]:
cubes_tensor = torch.tensor(cubes)

In [ ]:
cubes_rearrange =cubes[np.array(K_permutations[label])]

In [ ]:
print('label', np.array(K_permutations[label]), label)
return np.array(cubes)[np.array(K_permutations[label])], label

In [ ]:
import torch

# 嵌套列表
nested_list = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
]

# 将嵌套列表转换为 PyTorch 张量
nested_tensor = torch.tensor(nested_list)

# 打印结果
print(nested_tensor)